# 02 - Modelo Logístico

En este notebook se entrena un modelo base de **Regresión Logística** utilizando los datos ya procesados en `01_exploracion_datos_v2.ipynb`.


In [1]:
# Librerías principales
import pandas as pd
import numpy as np

# Librerías de modelado
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score

# Configuración general
pd.set_option('display.max_columns', None)


In [4]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

rfm_clients_v3 = pd.read_csv('../data/raw/clients_202510280926.csv')

# solo para probar estructura
print(rfm_clients_v3.head())


                              client_id      first_name last_name gender  age  \
0  21e5c13d-1c9a-4d00-9164-b72302d5edef         Tristán     Rojas      M   42   
1  36e48bdd-db11-4abe-9526-cfc90e68924d           Óscar  Barranco      M   58   
2  145c22df-3579-412e-bc12-b4fce70abaf3         Macario    Querol      M   28   
3  90c4a925-e51f-4dac-9193-2d9aec97a472          Ramiro   Sanchez      M   19   
4  853f711a-4c36-40b4-b5d0-6207152cd793  Juan Francisco      Flor      M   36   

                               email      city               address  \
0        tristán.rojas13@example.com    Rivera  Bulevar Artigas 3757   
1       óscar.barranco13@example.com     Salto  Bulevar Artigas 3911   
2       macario.querol67@example.com    Rivera        Av. Rivera 206   
3       ramiro.sanchez53@example.com     Salto  Av. 18 de Julio 6324   
4  juan francisco.flor68@example.com  Mercedes  Av. 18 de Julio 6301   

   postal_code  country citizenship document_type document_number  \
0        28

In [5]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

rfm_clients_v3 = pd.read_csv('../data/raw/clients_202510280926.csv')

# solo para probar estructura
print(rfm_clients_v3.head())


                              client_id      first_name last_name gender  age  \
0  21e5c13d-1c9a-4d00-9164-b72302d5edef         Tristán     Rojas      M   42   
1  36e48bdd-db11-4abe-9526-cfc90e68924d           Óscar  Barranco      M   58   
2  145c22df-3579-412e-bc12-b4fce70abaf3         Macario    Querol      M   28   
3  90c4a925-e51f-4dac-9193-2d9aec97a472          Ramiro   Sanchez      M   19   
4  853f711a-4c36-40b4-b5d0-6207152cd793  Juan Francisco      Flor      M   36   

                               email      city               address  \
0        tristán.rojas13@example.com    Rivera  Bulevar Artigas 3757   
1       óscar.barranco13@example.com     Salto  Bulevar Artigas 3911   
2       macario.querol67@example.com    Rivera        Av. Rivera 206   
3       ramiro.sanchez53@example.com     Salto  Av. 18 de Julio 6324   
4  juan francisco.flor68@example.com  Mercedes  Av. 18 de Julio 6301   

   postal_code  country citizenship document_type document_number  \
0        28

In [13]:
df = rfm_clients_v3.copy()
df['registration_date'] = pd.to_datetime(df['registration_date'], errors='coerce')
df['last_seen'] = pd.to_datetime(df['last_seen'], errors='coerce')
today = pd.Timestamp('today').normalize()
df['tenure_days'] = (today - df['registration_date']).dt.days
df['days_since_last_seen'] = (today - df['last_seen']).dt.days
df['churn'] = (df['days_since_last_seen'] >= 60).astype(int)

numeric_cols = ['age','tenure_days','days_since_last_seen']
categorical_cols = ['gender','city']

base = df[numeric_cols + categorical_cols + ['churn']].dropna()
X = pd.get_dummies(base[numeric_cols + categorical_cols], drop_first=True)
y = base['churn']


In [14]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

scaler = StandardScaler()
X[numeric_cols] = scaler.fit_transform(X[numeric_cols])

X_train_v3, X_test_v3, y_train_v3, y_test_v3 = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

X_train_v3.shape, X_test_v3.shape, y_train_v3.mean(), y_test_v3.mean()


((1885, 12), (472, 12), 1.0, 1.0)

In [16]:
y_train_v3.value_counts()


churn
1    1885
Name: count, dtype: int64

In [17]:
y_test_v3.value_counts()


churn
1    472
Name: count, dtype: int64

In [18]:
df['churn'] = (df['days_since_last_seen'] >= 30).astype(int)


In [19]:
base = df[numeric_cols + categorical_cols + ['churn']].dropna()


In [20]:
y_train_v3.value_counts()


churn
1    1885
Name: count, dtype: int64

In [23]:
df['days_since_last_seen'].describe()


count    2357.000000
mean      312.456937
std       215.290005
min        66.000000
25%       133.000000
50%       254.000000
75%       440.000000
max      1028.000000
Name: days_since_last_seen, dtype: float64

In [24]:
for t in [10, 20, 30, 45, 60, 90]:
    churn_temp = (df['days_since_last_seen'] >= t).astype(int)
    print(t, churn_temp.value_counts(normalize=True).to_dict())


10 {1: 1.0}
20 {1: 1.0}
30 {1: 1.0}
45 {1: 1.0}
60 {1: 1.0}
90 {1: 0.8790835808230801, 0: 0.12091641917691981}


In [26]:
df[['registration_date','last_seen']].agg(['min','max'])
df['days_since_last_seen'].describe()
for t in [60,75,90,120,180,365]:
    c = (df['days_since_last_seen'] >= t).astype(int)
    print(t, c.value_counts().to_dict())


60 {1: 2357}
75 {1: 2233, 0: 124}
90 {1: 2072, 0: 285}
120 {1: 1847, 0: 510}
180 {1: 1513, 0: 844}
365 {0: 1551, 1: 806}


In [27]:
df['churn'] = (df['days_since_last_seen'] >= 90).astype(int)
df['churn'].value_counts()


churn
1    2072
0     285
Name: count, dtype: int64

In [28]:
numeric_cols = ['age','tenure_days','days_since_last_seen']
categorical_cols = ['gender','city']

base = df[numeric_cols + categorical_cols + ['churn']].dropna()
X = pd.get_dummies(base[numeric_cols + categorical_cols], drop_first=True)
y = base['churn']

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X[numeric_cols] = scaler.fit_transform(X[numeric_cols])

from sklearn.model_selection import train_test_split
X_train_v3, X_test_v3, y_train_v3, y_test_v3 = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

y_train_v3.value_counts(), y_test_v3.value_counts()


(churn
 1    1657
 0     228
 Name: count, dtype: int64,
 churn
 1    415
 0     57
 Name: count, dtype: int64)

In [29]:
from sklearn.linear_model import LogisticRegression

log_model = LogisticRegression(solver='lbfgs', max_iter=1000, class_weight='balanced', random_state=42)
log_model.fit(X_train_v3, y_train_v3)


LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42)

In [32]:
X_train_v3.to_csv('../data/processed/X_train_v3.csv', index=False)
X_test_v3.to_csv('../data/processed/X_test_v3.csv', index=False)
y_train_v3.to_csv('../data/processed/y_train_v3.csv', index=False)
y_test_v3.to_csv('../data/processed/y_test_v3.csv', index=False)


**Modelo entrenado con Dataset RFM**

In [33]:
import pandas as pd, numpy as np
X_train_v3 = pd.read_csv('../data/processed/X_train_v3.csv')
X_test_v3  = pd.read_csv('../data/processed/X_test_v3.csv')
y_train_v3 = pd.read_csv('../data/processed/y_train_v3.csv').squeeze()
y_test_v3  = pd.read_csv('../data/processed/y_test_v3.csv').squeeze()

X_train_v3.shape, X_test_v3.shape, y_train_v3.value_counts().to_dict(), y_test_v3.value_counts().to_dict()


((1885, 12), (472, 12), {1: 1657, 0: 228}, {1: 415, 0: 57})

In [34]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, average_precision_score

log_model = LogisticRegression(solver='lbfgs', max_iter=1000, class_weight='balanced', random_state=42)
log_model.fit(X_train_v3, y_train_v3)

y_pred = log_model.predict(X_test_v3)
y_proba = log_model.predict_proba(X_test_v3)[:,1]

print(f'Accuracy: {accuracy_score(y_test_v3, y_pred):.3f}')
print(f'ROC AUC: {roc_auc_score(y_test_v3, y_proba):.3f}')
print(f'Average Precision: {average_precision_score(y_test_v3, y_proba):.3f}')
print(classification_report(y_test_v3, y_pred, digits=3))
confusion_matrix(y_test_v3, y_pred)


Accuracy: 0.898
ROC AUC: 0.999
Average Precision: 1.000
              precision    recall  f1-score   support

           0      0.543     1.000     0.704        57
           1      1.000     0.884     0.939       415

    accuracy                          0.898       472
   macro avg      0.771     0.942     0.821       472
weighted avg      0.945     0.898     0.910       472



array([[ 57,   0],
       [ 48, 367]])

In [36]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, average_precision_score, classification_report, confusion_matrix

tree_model = DecisionTreeClassifier(max_depth=5, random_state=42, class_weight='balanced')
tree_model.fit(X_train_v3, y_train_v3)

y_pred_tree = tree_model.predict(X_test_v3)
y_proba_tree = tree_model.predict_proba(X_test_v3)[:,1]

print(f'Accuracy: {accuracy_score(y_test_v3, y_pred_tree):.3f}')
print(f'ROC AUC: {roc_auc_score(y_test_v3, y_proba_tree):.3f}')
print(f'Average Precision: {average_precision_score(y_test_v3, y_proba_tree):.3f}')
print(classification_report(y_test_v3, y_pred_tree, digits=3))
print(confusion_matrix(y_test_v3, y_pred_tree))


Accuracy: 1.000
ROC AUC: 1.000
Average Precision: 1.000
              precision    recall  f1-score   support

           0      1.000     1.000     1.000        57
           1      1.000     1.000     1.000       415

    accuracy                          1.000       472
   macro avg      1.000     1.000     1.000       472
weighted avg      1.000     1.000     1.000       472

[[ 57   0]
 [  0 415]]


In [37]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42, class_weight='balanced')
rf_model.fit(X_train_v3, y_train_v3)

y_pred_rf = rf_model.predict(X_test_v3)
y_proba_rf = rf_model.predict_proba(X_test_v3)[:,1]

print(f'Accuracy: {accuracy_score(y_test_v3, y_pred_rf):.3f}')
print(f'ROC AUC: {roc_auc_score(y_test_v3, y_proba_rf):.3f}')
print(f'Average Precision: {average_precision_score(y_test_v3, y_proba_rf):.3f}')
print(classification_report(y_test_v3, y_pred_rf, digits=3))
print(confusion_matrix(y_test_v3, y_pred_rf))


Accuracy: 1.000
ROC AUC: 1.000
Average Precision: 1.000
              precision    recall  f1-score   support

           0      1.000     1.000     1.000        57
           1      1.000     1.000     1.000       415

    accuracy                          1.000       472
   macro avg      1.000     1.000     1.000       472
weighted avg      1.000     1.000     1.000       472

[[ 57   0]
 [  0 415]]
